In [1]:
import os
import gc

import numpy as np
import pandas as pd

from utils_prediction.dataloader.mimic4 import dataloader

In [2]:
base_fpath = '/hpf/projects/lsung/projects/mimic4ds/Experiments/baseline/data'

In [3]:
timebins = ['_0_0.167','_-7_0','_-30_-7','_-180_-30','-365000_-180']

feature_tags = [
    ['diag','count'],
    ['hcpcs','count'],
    ['lab','count'],
    ['presc','count'],
    ['proc','count'],
    ['icucharts','measurement'],
    ['lab','measurement']
]

config = {
    'features_fpath':base_fpath,
    'features_ftype':'parquet',
    'verbose':False,
    'label_col':'label',
    'id_col':'subject_id',
    'group':'2008 - 2010'
}

In [ ]:
## grab all feature labels
all_cols = {}

for task in ['mortality','longlos','sepsis','invasivevent']:
    config['analysis_id'] = task

    data = dataloader(**config).load_features()
    cols = data.features.columns
    df = pd.DataFrame(columns=['task','tag','timebins','count'])
    c = 0
    for timebin in timebins:
        for t,tag in enumerate(feature_tags):
            c+=1
            df.loc[c,:] = [
                task,
                '_'.join(feature_tags[t]),
                timebin,
                len([
                    x for x in cols
                    if timebin in x and
                    tag[0] in x and
                    tag[1] in x
                ])
            ]
    df.to_csv(f'artifacts/{task}_features.csv',index=False)

In [3]:
## join dfs
df = pd.concat((
    pd.read_csv(f'artifacts/{x}_features.csv')
    for x in ['mortality','longlos','sepsis','invasivevent']
), axis=0)

df = df.pivot(
    index = ['task','timebins'], 
    columns = 'tag', 
    values = 'count'
)
df = df.reindex(labels = ['mortality','longlos','invasivevent','sepsis'], level=0)
df = df[['diag_count','proc_count','hcpcs_count','presc_count','lab_count','lab_measurement','icucharts_measurement']]

In [8]:
# total number of features
df.groupby('task').sum().sum(axis=1)

task
mortality       48423
longlos         48423
invasivevent    44907
sepsis          40691
dtype: int64

In [9]:
df

tag                        diag_count  proc_count  hcpcs_count  presc_count  \
task         timebins                                                         
mortality    -365000_-180        2189         545           42         2037   
             _-180_-30           5450        1727          198         3424   
             _-30_-7             4534        1402          121         3390   
             _-7_0               3262         900           68         3657   
             _0_0.167               0           0            0         3154   
longlos      -365000_-180        2189         545           42         2037   
             _-180_-30           5450        1727          198         3424   
             _-30_-7             4534        1402          121         3390   
             _-7_0               3262         900           68         3657   
             _0_0.167               0           0            0         3154   
invasivevent -365000_-180        1943         467           34         1885   
             _-180_-30           4922        1524          176         3262   
             _-30_-7             4091        1190          106         3221   
             _-7_0               2909         765           58         3493   
             _0_0.167               0           0            0         2985   
sepsis       -365000_-180        1581         362           31         1635   
             _-180_-30           4269        1310          151         3031   
             _-30_-7             3459         961           78         3007   
             _-7_0               2473         614           42         3413   
             _0_0.167               0           0            0         2792   

tag                        lab_count  lab_measurement  icucharts_measurement  
task         timebins                                                         
mortality    -365000_-180        556             1071                      0  
             _-180_-30           729             1395                      0  
             _-30_-7             714             1326                      0  
             _-7_0               743             1389                      0  
             _0_0.167            653             1257                   2490  
longlos      -365000_-180        556             1071                      0  
             _-180_-30           729             1395                      0  
             _-30_-7             714             1326                      0  
             _-7_0               743             1389                      0  
             _0_0.167            653             1257                   2490  
invasivevent -365000_-180        537             1026                      0  
             _-180_-30           717             1365                      0  
             _-30_-7             698             1287                      0  
             _-7_0               731             1374                      0  
             _0_0.167            634             1212                   2295  
sepsis       -365000_-180        482              930                      0  
             _-180_-30           690             1323                      0  
             _-30_-7             688             1287                      0  
             _-7_0               718             1335                      0  
             _0_0.167            612             1146                   2271